In [56]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
train_tape = "Z:/rppg/ccnu_datatape_32x72x72_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_32x72x72_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

def to_tf_ep(datatape, dtype=tf.float16):
    def _():
        for i, j in datatape:
            j = tf.concat([j[1:]-j[:-1], [0]], axis=0)
            yield i, j
    return tf.data.Dataset.from_generator(lambda :_(), output_types=(dtype, dtype), output_shapes=(datatape.shape, datatape.shape[:1]))

#train, valid = to_tf_ep(train).cache(f'{tmp}/ep'), to_tf_ep(valid).cache(f'{tmp}/ep')
train, valid = to_tf_ep(train), to_tf_ep(valid)

In [39]:
ep = EP(n=32)
ep.compile(optimizer='Nadam', loss='mse')
ep.build(input_shape=(None, 72, 72, 3))
print(f'Flops per frame:{get_flops(ep, input_sig=[tf.TensorSpec([32, 72, 72, 3])])/32:.0f}')
ep.summary()

Flops per frame:230341239
Model: "ep_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tsm_31 (TSM)                 multiple                  0         
_________________________________________________________________
conv2d_186 (Conv2D)          multiple                  896       
_________________________________________________________________
conv2d_187 (Conv2D)          multiple                  9248      
_________________________________________________________________
conv2d_188 (Conv2D)          multiple                  18496     
_________________________________________________________________
conv2d_189 (Conv2D)          multiple                  36928     
_________________________________________________________________
conv2d_190 (Conv2D)          multiple                  33        
_________________________________________________________________
attention_mask_31 (Attention multip

In [ ]:
ep.fit(train.shuffle(2000), validation_data=valid, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/EfficientPhys_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
ep.load_weights('../weights/EfficientPhys_CCNU.h5')

In [42]:
ep.load_weights('../weights/EfficientPhys_CCNU.h5')

In [47]:
eval_on_dataset(test_set_CCNU, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_CCNU.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_CCNU.h5')

100%|██████████| 179/179 [11:50<00:00,  3.97s/it]


{'Sliding window': {'MAE': 1.097, 'RMSE': 3.963, 'R': 0.92474},
 'Whole video': {'MAE': 0.479, 'RMSE': 0.796, 'R': 0.99644}}

In [48]:
get_metrics_HRV('../results/EfficientPhys_CCNU_CCNU.h5')

{'SDNN': {'MAE': 37.548, 'RMSE': 48.659, 'R': 0.45769}}

In [49]:
eval_on_dataset(test_set_CCNU_rPPG, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_CCNU_rPPG.h5')

100%|██████████| 54/54 [01:08<00:00,  1.27s/it]


{'Sliding window': {'MAE': 1.164, 'RMSE': 4.489, 'R': 0.89927},
 'Whole video': {'MAE': 0.473, 'RMSE': 0.786, 'R': 0.99639}}

In [50]:
get_metrics_HRV('../results/EfficientPhys_CCNU_CCNU_rPPG.h5')

{'SDNN': {'MAE': 47.662, 'RMSE': 56.921, 'R': 0.33198}}

In [57]:
eval_on_dataset(test_set_PURE, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_PURE.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_PURE.h5')

100%|██████████| 59/59 [00:43<00:00,  1.34it/s]


{'Sliding window': {'MAE': 3.155, 'RMSE': 10.898, 'R': 0.89357},
 'Whole video': {'MAE': 1.39, 'RMSE': 5.98, 'R': 0.96767}}

In [52]:
get_metrics_HRV('../results/EfficientPhys_CCNU_PURE.h5')

x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\ma\core.py:5288: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
x:\Users\wkg\anaconda3\envs\dl\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


{'SDNN': {'MAE': 31.862, 'RMSE': 49.01, 'R': 0.38763}}

In [53]:
eval_on_dataset(test_set_UBFC_rPPG2, ep, 32, (72, 72), step=1, batch=0, save='../results/EfficientPhys_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC, cumsum=True)
get_metrics('../results/EfficientPhys_CCNU_UBFC.h5')

100%|██████████| 42/42 [00:29<00:00,  1.40it/s]


{'Sliding window': {'MAE': 1.466, 'RMSE': 3.84, 'R': 0.97522},
 'Whole video': {'MAE': 1.077, 'RMSE': 1.465, 'R': 0.99677}}

In [54]:
get_metrics_HRV('../results/EfficientPhys_CCNU_UBFC.h5')

{'SDNN': {'MAE': 9.762, 'RMSE': 15.048, 'R': 0.82734}}